In [1]:
# setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import sys
import http.client, urllib.request, urllib.parse, urllib.error, base64
import config

from sklearn.model_selection import GridSearchCV

root= r"C:/Users/Jennah/Desktop/Code/machine-learning-final"
inp= os.join(root, "data", "3_clean")
interm= os.join(root, "data", "2_intermediate")

plots= os.join(root, "code", "plots")

In [2]:
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=570719947456-53haph7mjc317oel1ujh0kmjbfd499jf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [3]:
# files in clean data google drive
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1jxSI_nq32-W9kPHKvQkzuNGJ5aIBD2j2' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

Title: analysis_box.csv, ID: 1Vzjvta98IG9Hg4SfAGcd8JD8FaF3IXwy
Title: analysis_bbl.csv, ID: 1yYX-ROr1c7fzcQiG6NZ1sh7Ko5gqr_4G
Title: analysis_bbl_noacs.csv, ID: 1h-tDpBSslfzqzsJe1ny_ujDnr68RKry3
Title: borobox_convex_hull.geojson, ID: 1SzBNj3Ba-v9MEwuCkZT9e9iAwVYUqx8r


In [4]:
# load codebook        
codebk = drive.CreateFile({'id': '1Lf7kV9_R-caFZV6_l8hMxAVjFSVFhvYl'})
codebk.GetContentFile('codebook.xlsx')  

codebk= pd.read_excel('codebook.xlsx', sheet_name = "predictors")
codebk.head()

C:\Users\Jennah\anaconda3\envs\geo_env\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Variable,Description,Source,Notes,Include_BBL,Include_Box
0,geofips,census tract geo identifier,Census ACS,NaN,0.0,0.0
1,geofips_recode,"census tract geo identifier, changes county to...",Census ACS,NaN,0.0,0.0
2,totpop,total population,Census ACS,NaN,1.0,1.0
3,popdens,total population per square mile,Census ACS,NaN,1.0,1.0
4,areasqmile,area,Census ACS,NaN,1.0,1.0


In [5]:
predictors_full = codebk.loc[codebk["Include_Box"] == 1, "Variable"].apply(lambda x: x.strip())
predictors_full

2            totpop
3           popdens
4        areasqmile
5             males
6           females
           ...     
164    bldgclass_A2
165    bldgclass_A0
166    bldgclass_A6
167    bldgclass_A8
168    bldgclass_CM
Name: Variable, Length: 150, dtype: object

## Load analysis file

In [6]:
df_box = drive.CreateFile({'id': '1Vzjvta98IG9Hg4SfAGcd8JD8FaF3IXwy'})
df_box.GetContentFile('analysis_box.csv')  

df_box= pd.read_csv('analysis_box.csv').drop(["Unnamed: 0"], axis = 1)
df_box.head()

,borobox,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,...,pctmvdsamecounty,pctmvdsdiffcounty,pctmvdsdiffstate,pctmvdabroad,pctpoprenterhh,engines_assigned_quantity,ladders_assigned_quantity,dispatch_response_seconds_qy,num_fire_ev,avg_fire_ev_yr
0,B0020,3457.956522,9239.391304,1.173913,3.521739,9.000000,9.043478,3.718826e+04,8.482943e+05,2.937391e+03,...,4.279476,13.100437,3.580786,0.0,0.432729,3.500000,2.333333,32.666667,6,1.200000
1,B0023,26081.875000,209572.375000,1.000000,18.750000,229.750000,231.125000,1.322048e+06,1.219295e+07,2.589945e+06,...,4.279476,13.100437,3.580786,0.0,0.432729,3.000000,2.000000,42.333333,3,3.000000
2,B0024,2754.847826,4832.108696,1.086957,3.217391,5.695652,5.847826,4.959654e+04,5.346010e+05,4.643715e+04,...,4.279476,13.100437,3.580786,0.0,0.432729,3.200000,2.040000,36.880000,25,1.923077
3,B0025,61653.000000,85292.250000,1.250000,10.750000,101.500000,102.000000,8.558175e+05,3.580575e+06,2.746800e+06,...,4.279476,13.100437,3.580786,0.0,0.432729,2.400000,1.600000,15.600000,5,1.666667
4,B0027,3888.574713,8242.965517,1.126437,3.183908,7.080460,7.402299,4.881548e+04,6.619610e+05,1.669940e+05,...,4.279476,13.100437,3.580786,0.0,0.432729,3.545455,2.340909,32.181818,44,2.933333


In [7]:
assert df_box.dropna().shape[0] == df_box.shape[0]

AssertionError: 

In [ ]:
df_box.columns
predictors = ["lotarea", "bldgarea", "numbldgs", "numfloors", "unitsres", "unitstotal", "assessland",
             "assesstot", "exempttot", "yearbuilt", "holc_AB", "holc_CD", "holc_D",
             "tot_hpd_vio", "hpd_fire_vio", "hpd_b_c_vio",
             "viol_count_per_unit", "hpd_fire_vio_per_unit", "hpd_b_c_vio_per_unit",
             "AL", "NB", "average_energy_usage", "average_energy_usage_mi"]

predictors_acs= df_box.columns[df_box.columns.get_loc("totpop"):df_box.columns.get_loc("pctrentocch") + 1].tolist()
landusecols= [x for x in df_box.columns if x in 'landuse_']
bldgclasscols= [x for x in df_box.columns if x in 'bldgclass_']

predictors_full= predictors + predictors_acs  + landusecols + bldgclasscols
predictors_full;

In [ ]:
outcomes= ["dispatch_response_seconds_qy", "num_fire_ev", "avg_fire_ev_yr"]
df_box[outcomes].head()

# (0) Prep data

In [ ]:
df_box.describe()

In [ ]:
df_box.loc[df_box["avg_fire_ev_yr"]==0].shape[0]/df_box.shape[0]

In [ ]:
# assert no missingness
assert (df_box.isna().sum(axis = 0).sum() == 0)

In [ ]:
df_box_clean= df_box.dropna(how = "any")
df_box_clean.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

X = df_box_clean.loc[:, predictors_full]
y = df_box_clean.loc[X.index, outcomes]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=999)

### Running LASSO to determine important variables

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(X_scaled, y, test_size=0.3, random_state=999)

In [ ]:
from sklearn import linear_model

coef_comp= pd.DataFrame(X.columns, columns = ['Features'])
coef_comp

param_grid= {'alpha':[.00001, 0.0001,0.001, 0.01, 0.5, 1, 1.5, 2]}
clf= linear_model.Lasso(fit_intercept = True)
gr = GridSearchCV(clf, param_grid=param_grid, n_jobs=-1)
rs = gr.fit(X_train_l, y_train_l.iloc[:, 1])

print(rs.best_params_)
print(rs.best_score_)

clf= linear_model.Lasso(fit_intercept = True, alpha = rs.best_params_['alpha'])
clf.fit(X_train_l, y_train_l.iloc[:, 1])
coef_comp['coef_alpha_0.5']= clf.coef_

In [ ]:
clf= linear_model.Lasso(fit_intercept = True, alpha = 3)
clf.fit(X_train_l, y_train_l.iloc[:, 1])

# feature selection from LASSOs
# fewer features chosen
coef_comp['coef_alpha_3']= clf.coef_
lasso_coef= coef_comp.loc[coef_comp['coef_alpha_0.5'] != 0].sort_values('coef_alpha_0.5', ascending = False)
lasso_coef

# (1) Decision Trees and Random Forest Models
* All are negative, suggests the model isn't performing very well

In [ ]:
from sklearn.metrics import mean_squared_error
for c in range(3):
    print("\nDecision tree for", outcomes[c])
    print()
    
    dt1 = DecisionTreeRegressor(max_depth=10, random_state = 99)
    dt1.fit(X_train, y_train.iloc[:, c])

    dt2 = DecisionTreeRegressor(max_depth=15, random_state = 99)
    dt2.fit(X_train, y_train.iloc[:, c])

    y_pred1= dt1.predict(X_test)
    y_pred2= dt2.predict(X_test)

    print("R2 for model 1:", dt1.score(X_test, y_test.iloc[:, c]))
    print("R2 for model 2:", dt2.score(X_test, y_test.iloc[:, c]))
    
    print("\nMSE for model 1:", mean_squared_error(y_test.iloc[:, c], y_pred1))
    print("MSE for model 2:", mean_squared_error(y_test.iloc[:, c], y_pred2))

## Tuning
* There is a higher R2 for num_fire_ev and avg_fire_ev_yr
* Seems to do well with a simpler model

In [ ]:
# use max_depth to control the complexity of the tre
param_grid = {'max_depth':np.arange(1, 20)}
bp= []
s= []
for c in range(3):
    dt=DecisionTreeRegressor()
    gr=GridSearchCV(dt,param_grid=param_grid)
    rs=gr.fit(X_train,y_train.iloc[:, c])
    
    bp+= [rs.best_params_]
    s+= [rs.score(X_test, y_test.iloc[:, c])]
    
    print("\nTuning decision tree for", outcomes[c])
    print(rs.best_params_)
    print(rs.score(X_test, y_test.iloc[:, c]))

In [ ]:
# the R2 is pretty low...
print(bp)
print(s)

In [ ]:
## can we visualize the decision tree?
## for num_fire_ev
from sklearn import tree
from IPython.display import Image  
import pydotplus

dt = DecisionTreeRegressor(max_depth= 4, random_state = 99) #bp[1]['max_depth']) # since max depth is optimal for 3, use that
dt.fit(X_train,y_train.iloc[:, 1])

thestring = tree.export_graphviz(dt, out_file=None,  
                         feature_names=X_train.columns.values, 
                         filled=True, rounded=True,  
                         special_characters=True,impurity=False).replace(" ;\n}", " ;}")
graph = pydotplus.graph_from_dot_data(thestring);
graph.write_png(os.join(plots,'num_fire_ev_dt.png'))
Image(graph.create_png())

## Feature Importance

In [ ]:
dt = DecisionTreeRegressor(random_state = 99, max_depth= 2) #rs.best_params_['max_depth'])
dt.fit(X_train, y_train)
Feature_importance=pd.DataFrame([list(X_train.columns),list(dt.feature_importances_)]).T
Feature_importance.columns=["variables","importance"]

# list the top 10 most importnat features in order (using max_depth = 10)
ft1= Feature_importance.sort_values(by="importance",ascending=False).iloc[:10,:]
ft1

In [ ]:
dt = DecisionTreeRegressor(max_depth=4, random_state = 99)
dt.fit(X_train, y_train)
Feature_importance=pd.DataFrame([list(X_train.columns),list(dt.feature_importances_)]).T
Feature_importance.columns=["variables","importance"]

# list the top 10 most important features in order (using max_Depth = 4)
ft2= Feature_importance.sort_values(by="importance",ascending=False).iloc[:10,:]
ft2

In [ ]:
plt.figure(figsize=(15, 5), dpi=80)
plt.bar(ft2.sort_values(by="importance",ascending=False).iloc[:,0],
        ft2.sort_values(by="importance",ascending=False).iloc[:,1])
plt.title("Feature Importance");

## Random Forests
* Only outcome is number of num_fire_ev

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=30, n_jobs=-1, max_depth = 4, random_state = 99)
rf.fit(X_train, y_train.iloc[:, 1])
y_pred=rf.predict(X_test)
print("R2 with random forest:", rf.score(X_test, y_test.iloc[:, 1]))

In [ ]:
param_grid = {'max_depth':range(1,30)}
rf = RandomForestRegressor(n_jobs=-1, n_estimators=30, random_state = 99)
gs = GridSearchCV(rf,param_grid=param_grid)
rs = gs.fit(X_train,y_train.iloc[:, 1])
y_pred= rs.predict(X_test)

print(rs.best_params_)
print("R2 with best paramters:", rs.score(X_test, y_test.iloc[:, 1]))

### CMM Method and Other Ensemble Methods
* [From Domingos, 1998](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.40.2710&rep=rep1&type=pdf)
* Not much of an increase in accuracy compared to decision tree model

In [ ]:
rf = RandomForestRegressor(n_jobs=-1, n_estimators=30, random_state = 99, max_depth = 6) #rs.best_params_['max_depth'])
rf.fit(X_train,y_train.iloc[:, 1])

# randomly generate a sample of xs
X_rand= pd.DataFrame({x:X_train[x].sample(n = round(0.5*X_train.shape[0])).reset_index(drop = True) for x in X_train.columns})
y_rand= rf.predict(X_rand)

In [ ]:
# create the cmm training datasets
X_train_cmm= pd.concat([X_rand, X_train], axis = 0, ignore_index = True)
y_train_cmm= pd.concat([round(pd.DataFrame(y_rand)), y_train.iloc[:, 1]], axis = 0, ignore_index = True)

In [ ]:
param_grid = {'max_depth':range(1,30)}
dt=DecisionTreeRegressor()
gr=GridSearchCV(dt,param_grid=param_grid)
rs=gr.fit(X_train_cmm, y_train_cmm)

print(rs.best_params_)
print(rs.score(X_test, y_test.iloc[:, 1]))

In [ ]:
## prediction
pred = rs.predict(X_test)
pred

In [ ]:
dt = DecisionTreeRegressor(max_depth= 3, random_state = 99) #rs.best_params_['max_depth']) # since max depth is optimal for 3, use that
dt.fit(X_train_cmm, y_train_cmm)

In [ ]:
Feature_importance=pd.DataFrame([list(X_train_cmm.columns),list(dt.feature_importances_)]).T
Feature_importance.columns=["variables","importance"]

# list the top 10 most importnat features in order (using max_depth = 10)
ft3= Feature_importance.sort_values(by="importance",ascending=False)
ft3.loc[ft3['importance']>0]

In [ ]:
## we can plot this
from sklearn import tree
from IPython.display import Image  
import pydotplus

thestring = tree.export_graphviz(dt, out_file=None,  
                         feature_names=X_train.columns.values, 
                         filled=True, rounded=True,  
                         special_characters=True,impurity=False).replace(" ;\n}", " ;}")
graph = pydotplus.graph_from_dot_data(thestring);
graph.write_png(os.join(plots,'num_fire_ev_cmm.png'))
Image(graph.create_png())

# (2) Bayes Nets

In [ ]:
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import K2Score, BicScore, BDeuScore
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
import matplotlib.pyplot as plt

In [ ]:
## need to discretize outcomes
df_box_disc= df_box.copy()

In [ ]:
# discretize each variable to four equal-frequency bins (quartiles)
for i in df_box_disc.columns[1:]:
    if len(df_box_disc.loc[:, i].unique()) > 5:
        plt.hist(df_box_disc.loc[:, i], bins = 100);
        plt.title("Histogram for {}".format(i))
        print(plt.show())
        df_box_disc.loc[:, i] = pd.qcut(df_box_disc.loc[:, i], q=4, labels=False, duplicates = 'drop')

In [ ]:
## important vars
impt_vars= outcomes[0:2] + ft3.loc[ft3['importance']>0, "variables"].tolist()
impt_vars

In [ ]:
# create test train split for testing DAG
train, test = train_test_split(df_box_disc.loc[:, impt_vars], test_size=0.3, random_state=999)

In [ ]:
# use hill climb search with BIC score, as in hw
hc = HillClimbSearch(train)
best_model = hc.estimate(scoring_method=BicScore(train))
print("\nDAG model edges:")
print(best_model.edges())

In [ ]:
print("DAG model nodes:")
print(best_model.nodes())

In [ ]:
# look at the CPD for the model
model = BayesianNetwork(best_model.edges())
model.fit(train.loc[:, impt_vars], estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=10)
for cpd in model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
for n in model.nodes():
    print(model.local_independencies(n))

In [ ]:
import networkx as nx
import pylab as plt
fig = plt.gcf()

values = ['red' if n in ['num_fire_ev', 'dispatch_response_seconds_qy'] else 'lightblue' for n in model.nodes()]
# use networkx to draw DAG
nx.draw(model, node_color=values, with_labels=True, font_color='black', node_size=200,font_size=10)

plt.figure(figsize = (12,10))
fig.savefig('dag_.png')

In [ ]:
pred = model.predict(test.loc[:, [n for n in model.nodes() if n not in ['num_fire_ev', 'dispatch_response_seconds_qy']]])
print(pred)

In [ ]:
for i in impt_vars[0:2]:
    print("\nOS accuracy for: {}".format(i))
    print((pred[i].reset_index(drop = True) == test[i].reset_index(drop = True)).mean())

In [ ]:
# prediction just for time to the fire
pred = model.predict(test.loc[:, [n for n in model.nodes() if n not in ['dispatch_response_seconds_qy']]], stochastic = True)
print(pred)

In [ ]:
print("\nOS accuracy time to the fire, includes num_fire_ev as predictor")
print((pred['dispatch_response_seconds_qy'].reset_index(drop = True) == test['dispatch_response_seconds_qy'].reset_index(drop = True)).mean())

### Re-run Bayes Net with Feature Selection from LASSO

In [ ]:
impt_vars= outcomes[0:2] + lasso_coef.loc[lasso_coef["coef_alpha_3"] != 0, "Features"].tolist()

In [ ]:
# create test train split for testing DAG
train, test = train_test_split(df_box_disc.loc[:, impt_vars], test_size=0.3, random_state=999)

In [ ]:
# use hill climb search with BIC score, as in hw
hc = HillClimbSearch(train)
best_model = hc.estimate(scoring_method=BicScore(train))
print("\nDAG model edges:")
print(best_model.edges())

In [ ]:
print("DAG model nodes:")
print(best_model.nodes())

In [ ]:
# look at the CPD for the model
model = BayesianNetwork(best_model.edges())
model.fit(train.loc[:, impt_vars], estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=10)
for cpd in model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
for n in model.nodes():
    print(model.local_independencies(n))

In [ ]:
import networkx as nx
import pylab as plt
fig = plt.gcf()

values = ['red' if n in ['num_fire_ev', 'dispatch_response_seconds_qy'] else 'lightblue' for n in model.nodes()]
# use networkx to draw DAG
nx.draw(model, node_color=values, with_labels=True, font_color='black', node_size=200,font_size=10)

plt.figure(figsize = (12,10))
fig.savefig('dag_.png')

In [ ]:
pred = model.predict(test.loc[:, [n for n in model.nodes() if n not in ['num_fire_ev', 'dispatch_response_seconds_qy']]])
print(pred)

In [ ]:
for i in impt_vars[0:2]:
    print("\nOS accuracy for: {}".format(i))
    print((pred[i].reset_index(drop = True) == test[i].reset_index(drop = True)).mean())

In [ ]:
# prediction just for time to the fire
pred = model.predict(test.loc[:, [n for n in model.nodes() if n not in ['dispatch_response_seconds_qy']]], stochastic = True)
print(pred)

In [ ]:
print("\nOS accuracy time to the fire, includes num_fire_ev as predictor")
print((pred['dispatch_response_seconds_qy'].reset_index(drop = True) == test['dispatch_response_seconds_qy'].reset_index(drop = True)).mean())

### PC Method for Causal Model calculations

In [ ]:
from pgmpy.estimators import PC
from pgmpy.estimators.CITests import chi_square

In [ ]:
## important vars
impt_vars= outcomes[0:2] + ft3.loc[ft3['importance']>0, "variables"].tolist()
impt_vars

In [ ]:
# create test train split for testing DAG
train, test = train_test_split(df_box_disc.loc[:, impt_vars], test_size=0.3, random_state=999)
train.head()

In [ ]:
pc_est = PC(train)

# Step 1: construct undirected skeleton
skel, separating_sets = pc_est.build_skeleton(significance_level=0.01)
print("Undirected edges: ", skel.edges())

In [ ]:
# Step 2: partially directed acyclic graph 
pdag = pc_est.skeleton_to_pdag(skel, separating_sets)
print("PDAG edges:       ", pdag.edges())

In [ ]:
fig = plt.gcf()

values = ['red' if n in ['num_fire_ev'] else 'lightblue' for n in pdag.nodes()]
# use networkx to draw DAG
nx.draw(pdag, node_color=values, with_labels=True, font_color='black', node_size=200,font_size=10)

plt.figure(figsize = (12,10))
fig.savefig('pdag_box.png')

In [ ]:
model = pc_est.estimate()
print("DAG edges:        ", model.edges())

In [ ]:
fig = plt.gcf()

values = ['red' if n in ['num_fire_ev'] else 'lightblue' for n in model.nodes()]
# use networkx to draw DAG
nx.draw(model, node_color=values, with_labels=True, font_color='black', node_size=200,font_size=10)

plt.figure(figsize = (12,10))
fig.savefig('model_pdag_est_box.png')